In [ ]:
%load_ext autoreload
%autoreload 2

# Get parent directory and add to sys.path
import os
import sys

parent_dir = os.path.dirname(os.getcwd())
sys.path.append(parent_dir)

# Require ipympl
%matplotlib widget 

In [ ]:
# MPC import
from LinearMPC.MPCVelControl import MPCVelControl
from PIControl.PIControl import PIControl
import numpy as np
from src.rocket import Rocket
from src.vel_rocket_vis import RocketVis, plot_static_states_inputs

rocket_obj_path = os.path.join(parent_dir, "Cartoon_rocket.obj")
rocket_params_path = os.path.join(parent_dir, "rocket.yaml")

In [ ]:

Ts = 0.05
sim_time = 30
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 0, 0, 0, 50, 50, 100])  # initial state
pos_target = np.array([0, 0, 10.0])

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
pos_controller = PIControl(pos_target)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, pos_control=pos_controller, method="nonlinear"
)

In [ ]:
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 0.5
vis.animate(
    t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=ref[:, :-1],
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
);

In [ ]:
# Print to know how big the constraint violation is
from LinearMPC.utils import ALPHA, BETA

print(f"Max alpha: {np.rad2deg(max(np.abs(x_cl[ALPHA])))}")
print(f"Max beta: {np.rad2deg(max(np.abs(x_cl[BETA])))}")



In [ ]:
from LinearMPC.utils import X, Y, Z, X_TO_STRING
import matplotlib.pyplot as plt

x_ids = np.array([X, Y, Z])
legend = X_TO_STRING[x_ids]
plt.figure()
plt.plot(t_cl, np.transpose(x_cl[x_ids]))
plt.title("Closed-loop states trajectory")
plt.legend(legend)
plt.xlabel("time [s]")
plt.ylabel("state value")
plt.show()

x_ids = np.array([ALPHA, BETA])
legend = X_TO_STRING[x_ids]
plt.figure()
plt.plot(t_cl, np.rad2deg(np.transpose(x_cl[x_ids])))
plt.title("Closed-loop states trajectory")
plt.legend(legend)
plt.xlabel("time [s]")
plt.ylabel("state value")
plt.show()

#### ======================================== For 5.1 ===========================================

In [ ]:
Ts = 0.05
sim_time = 15
H = 5.0
x0 = np.array([0, 0, 0, 0, 0, 0, 5, 5, 10, 0, 0, 1])  # initial state
x_target = np.zeros((12,))

rocket = Rocket(Ts=Ts, model_params_filepath=rocket_params_path)
mpc = MPCVelControl().new_controller(rocket, Ts, H)

# Static mass change and zero fuel rate
rocket.mass = 1.5
rocket.fuel_rate = 0.0
t_cl, x_cl, u_cl, t_ol, x_ol, u_ol, ref = rocket.simulate_control(
    mpc, sim_time, H, x0, x_target=x_target, method="nonlinear"
)

In [ ]:
vis = RocketVis(rocket, rocket_obj_path)
vis.anim_rate = 0.5
vis.animate(t_cl[:-1],
    x_cl[:, :-1],
    u_cl,
    Ref=ref[:, :-1],
    T_ol=t_ol[..., :-1],
    X_ol=x_ol,
    U_ol=u_ol,
);

In [ ]:
from LinearMPC.utils import VZ, P_AVG, U_TO_STRING
x_ids = np.array([VZ])
legend = X_TO_STRING[x_ids]
plt.figure()
plt.plot(t_cl, np.transpose(x_cl[x_ids]))
plt.title("Closed-loop trajectory")
plt.legend(legend)
plt.xlabel("time [s]")
plt.ylabel("state value")
plt.show()
u_ids = np.array([P_AVG])
legend = U_TO_STRING[u_ids]
plt.figure()
plt.plot(t_cl[:-1], np.transpose(u_cl[u_ids]))
plt.title("Closed-loop trajectory")
plt.legend(legend)
plt.xlabel("time [s]")
plt.ylabel("state value")
plt.show()